## <center>Electiva Aprendizaje automático</center>
## <center>MGTIC - MISC</center>
## <center>Facultad de Ingeniería</center>
## <center>Universidad de Nariño</center>

### Integrantes:
- Pablo Vaca
- Willmer Escobar
- Yamid Estrada

### Docente:
- Mateo Guerrero

In [40]:
#conda install -c conda-forge spacy
from ftfy import fix_encoding
import pandas as pd
import spacy
from sqlalchemy import create_engine
import re
import string
from ftfy import fix_encoding
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [41]:
DATABASES = {
       'db':{
        'NAME': 'bd_ml',
        'USER': 'postgres',
        'PASSWORD': 'postgres1',
        'HOST': 'localhost',
        'PORT': 5432,
    }
}

db= DATABASES['db']


prod_engine_string = "postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}".format(
    user = db['USER'],
    password = db['PASSWORD'],
    host = db['HOST'],
    port = db['PORT'],
    database = db['NAME'],
)

prod= create_engine(prod_engine_string)

In [131]:
sql='select id, descripcion from public."tpFinal_entrenamiento"'
#sql='select id, lugar from public."tpFinal_entrenamiento"'
datos=pd.read_sql(sql,prod)
datos

,id,lugar
0,1169,Flores
1,1886,Villa Crespo
2,1825,Caballito
3,1442,Boedo
4,12741,Recoleta
...,...,...
11995,6045,Almagro
11996,10862,Caballito
11997,12591,Caballito
11998,11598,Belgrano


In [132]:
display(datos.isnull().sum())

id       0
lugar    0
dtype: int64

# Función Convertir el texto en minúsculas, eliminar el texto entre corchetes, eliminar la puntuación '.


In [134]:
def texto_limpio(t):
    t=str(t)
    t=t.lower()
    t=re.sub('\[.*?¿\]\%', ' ', t)
    t=re.sub('[%s]' % re.escape(string.punctuation), ' ', t)
    t=re.sub('\w*\d\w*', '', t)
    t=re.sub('[‘’“”…«»]', '', t)
    t=re.sub('\n', ' ', t)
    t=fix_encoding(t)
    t=t.encode('ascii', 'ignore').decode('utf-8')
    return t

round = lambda x: texto_limpio(x)

datos['descripcion_limpio'] = datos.descripcion.apply(round)
#datos['lugar_limpio'] = datos.lugar.apply(round)
datos.head(50)

,id,lugar,lugar_limpio
0,1169,Flores,flores
1,1886,Villa Crespo,villa crespo
2,1825,Caballito,caballito
3,1442,Boedo,boedo
4,12741,Recoleta,recoleta
5,2462,Villa Crespo,villa crespo
6,13824,Floresta,floresta
7,814,San Cristobal,san cristobal
8,14097,Flores,flores
9,5132,Belgrano,belgrano


In [66]:
!python -m spacy download es_core_news_sm

[+] Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')


In [135]:
%%time
stemmer = SnowballStemmer('spanish')
nlp = spacy.load('es_core_news_sm')

def tokenizar_stem(text):
    text=text.lower()
    doc1 = nlp(text)
    words=[]
    for t in doc1:
        if  t.is_punct or  t.is_stop or t.is_space:
            continue
        words.append(stemmer.stem(str(t)))
    return ' '.join(words)
display(stemmer.stem('administrador'))
datos['tokenizar_stem'] = datos.apply(lambda row: tokenizar_stem(str(row['descripcion_limpio'])),axis=1)
#datos['tokenizar_stem'] = datos.apply(lambda row: tokenizar_stem(str(row['lugar_limpio'])),axis=1)

'administr'

Wall time: 1min 7s


In [136]:
def tokenizar_lema(text):
    text=text.lower()
    doc = nlp(text,disable = ['ner', 'parser'])
    words=[]
    for t in doc:
        if  t.is_punct or  t.is_stop or t.is_space:
                continue
        words.append(t.lemma_) 
    return ' '.join(words)

def tokenizar_lema(text):
    doc1 = nlp(text)
    words=[]
    for t in doc1:
        if  t.is_punct or  t.is_stop or t.is_space or len(str(t))<=3:
            continue
        if t.ent_type==0:
            words.append(str(t.lemma_).lower())
        else:
            words.append(str(t).lower())
    return ' '.join(words)


display(tokenizar_lema('administre los datos de bienestar familiar'))

'administrar dato bienestar familiar'

In [137]:
%%time
datos['tokenizar_lema'] = datos.apply(lambda row: tokenizar_lema(str(row['descripcion_limpio'])),axis=1)
#datos['tokenizar_lema'] = datos.apply(lambda row: tokenizar_lema(str(row['lugar_limpio'])),axis=1)

Wall time: 1min 7s


In [138]:
datos.head(50)

,id,lugar,lugar_limpio,tokenizar_stem,tokenizar_lema
0,1169,Flores,flores,flor,flor
1,1886,Villa Crespo,villa crespo,vill cresp,villa crespo
2,1825,Caballito,caballito,caballit,caballito
3,1442,Boedo,boedo,boed,boedir
4,12741,Recoleta,recoleta,recolet,recoletar
5,2462,Villa Crespo,villa crespo,vill cresp,villa crespo
6,13824,Floresta,floresta,florest,floresta
7,814,San Cristobal,san cristobal,san cristobal,cristobal
8,14097,Flores,flores,flor,flor
9,5132,Belgrano,belgrano,belgran,belgrano


In [139]:
modelo_tf_idf = TfidfVectorizer(max_df=0.6, min_df=2,
                     lowercase=True,
                     ngram_range = (1,1),
                     analyzer = 'word',   
                     max_features=100) 

matrix_tf = modelo_tf_idf.fit_transform(datos.tokenizar_lema.apply(lambda x: np.str_(x)))
datos_estructurado=pd.DataFrame(matrix_tf.toarray(),columns=modelo_tf_idf.get_feature_names(),index=datos.id)

In [140]:
datos_estructurado

,abasto,agronomia,almagro,avellanedar,balvanero,barraca,barrio,belgrano,boca,boedir,...,soho,soldati,telmo,tribunal,urquizo,velez,versall,viejo,villa,villar
id,,,,,,,,,,,,,,,,,,,,,
1169,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1886,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.575079,0.0
1825,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1442,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
12741,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6045,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
10862,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
12591,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [141]:
columnas=[]
for c in datos_estructurado.columns:
    columnas.append(c.replace(" ","_"))
columnas

['abasto',
 'agronomia',
 'almagro',
 'avellanedar',
 'balvanero',
 'barraca',
 'barrio',
 'belgrano',
 'boca',
 'boedir',
 'caballito',
 'canita',
 'capital',
 'castro',
 'centenario',
 'centro',
 'cha',
 'chacabucir',
 'chacaritar',
 'chico',
 'coghlar',
 'colegial',
 'congreso',
 'constitucion',
 'crespo',
 'cristobal',
 'devoto',
 'federal',
 'flor',
 'floresta',
 'hollywood',
 'linier',
 'lugano',
 'luro',
 'madero',
 'matadero',
 'microcentro',
 'mitre',
 'monserrat',
 'monte',
 'nicolos',
 'norte',
 'nunez',
 'once',
 'ortuzar',
 'palermir',
 'palermo',
 'parque',
 'paternal',
 'patricio',
 'pompeya',
 'puerto',
 'pueyrredon',
 'real',
 'recoletar',
 'retiro',
 'rita',
 'saavedra',
 'santo',
 'sarsfield',
 'soho',
 'soldati',
 'telmo',
 'tribunal',
 'urquizo',
 'velez',
 'versall',
 'viejo',
 'villa',
 'villar']

In [142]:
datos_estructurado.columns=columnas
datos_estructurado

,abasto,agronomia,almagro,avellanedar,balvanero,barraca,barrio,belgrano,boca,boedir,...,soho,soldati,telmo,tribunal,urquizo,velez,versall,viejo,villa,villar
id,,,,,,,,,,,,,,,,,,,,,
1169,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1886,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.575079,0.0
1825,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1442,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
12741,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6045,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
10862,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
12591,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [76]:
datos_estructurado[datos_estructurado.palermo>0]

,abasto,agronomia,almagro,avellanedar,balvanero,barraca,barrio,belgrano,boca,boedir,...,soho,soldati,telmo,tribunal,urquizo,velez,versall,viejo,villa,villar
id,,,,,,,,,,,,,,,,,,,,,
13662,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7724,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5989,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12074,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.770336,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5032,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.770336,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14520,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13216,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.770336,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6541,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [77]:
%%time
datos_estructurado.to_sql(name="desc_1word_train",con=prod,schema="public",if_exists = 'replace', method="multi",chunksize = 1000,index=True)
#datos_estructurado.to_sql(name="lugar_1word_train",con=prod,schema="public",if_exists = 'replace', method="multi",chunksize = 1000,index=True)

Wall time: 18.4 s


In [143]:
sql='select id, descripcion from public."tpFinal_clasificacion"'
#sql='select id, lugar from public."tpFinal_clasificacion"'
datos_clas=pd.read_sql(sql,prod)
datos_clas

,id,lugar
0,13854,Boedo
1,622,Villa Lugano
2,7931,Palermo
3,4128,Villa Crespo
4,1379,Belgrano
...,...,...
2995,4446,Belgrano
2996,237,Parque Chas
2997,9979,Monserrat
2998,9770,Congreso


In [144]:
%%time
datos_clas['descripcion_limpio'] = datos_clas.descripcion.apply(round)
datos_clas['tokenizar_lema'] = datos_clas.apply(lambda row: tokenizar_lema(str(row['descripcion_limpio'])),axis=1)

#datos_clas['lugar_limpio'] = datos_clas.lugar.apply(round)
#datos_clas['tokenizar_lema'] = datos_clas.apply(lambda row: tokenizar_lema(str(row['lugar_limpio'])),axis=1)

matrix_tf_clas = modelo_tf_idf.transform(datos_clas.tokenizar_lema.apply(lambda x: np.str_(x)))
datos_estructurado_clas=pd.DataFrame(matrix_tf_clas.toarray(),columns=modelo_tf_idf.get_feature_names(),index=datos_clas.id)

Wall time: 14.9 s


In [145]:
datos_estructurado_clas

,abasto,agronomia,almagro,avellanedar,balvanero,barraca,barrio,belgrano,boca,boedir,...,soho,soldati,telmo,tribunal,urquizo,velez,versall,viejo,villa,villar
id,,,,,,,,,,,,,,,,,,,,,
13854,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
622,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.409701,0.0
7931,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.575079,0.0
1379,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
9979,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [146]:
columnas=[]
for c in datos_estructurado_clas.columns:
    columnas.append(c.replace(" ","_"))
columnas

['abasto',
 'agronomia',
 'almagro',
 'avellanedar',
 'balvanero',
 'barraca',
 'barrio',
 'belgrano',
 'boca',
 'boedir',
 'caballito',
 'canita',
 'capital',
 'castro',
 'centenario',
 'centro',
 'cha',
 'chacabucir',
 'chacaritar',
 'chico',
 'coghlar',
 'colegial',
 'congreso',
 'constitucion',
 'crespo',
 'cristobal',
 'devoto',
 'federal',
 'flor',
 'floresta',
 'hollywood',
 'linier',
 'lugano',
 'luro',
 'madero',
 'matadero',
 'microcentro',
 'mitre',
 'monserrat',
 'monte',
 'nicolos',
 'norte',
 'nunez',
 'once',
 'ortuzar',
 'palermir',
 'palermo',
 'parque',
 'paternal',
 'patricio',
 'pompeya',
 'puerto',
 'pueyrredon',
 'real',
 'recoletar',
 'retiro',
 'rita',
 'saavedra',
 'santo',
 'sarsfield',
 'soho',
 'soldati',
 'telmo',
 'tribunal',
 'urquizo',
 'velez',
 'versall',
 'viejo',
 'villa',
 'villar']

In [147]:
datos_estructurado_clas.columns=columnas
datos_estructurado_clas

,abasto,agronomia,almagro,avellanedar,balvanero,barraca,barrio,belgrano,boca,boedir,...,soho,soldati,telmo,tribunal,urquizo,velez,versall,viejo,villa,villar
id,,,,,,,,,,,,,,,,,,,,,
13854,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
622,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.409701,0.0
7931,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4128,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.575079,0.0
1379,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4446,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
237,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
9979,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [272]:
datos_estructurado_clas[datos_estructurado_clas.palermo>0]

,000_departamentir,000_departamento,000_estrenar,000_excelente,000m2_terreno,000subsuelo_cubierto,10_piso,100_metro,100_pesos,100_publicado,...,vivienda_apto,waimar_propiedad,zarlenguir_inmobiliaria,zona_belgrano,zona_comercial,zona_palermo,zona_residencial,zona_tranquilo,zona_villa,él_cucicba
id,,,,,,,,,,,,,,,,,,,,,
14894,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
4324,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
12480,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
7589,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
11406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
4896,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
14057,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0
10444,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.388326,0.0,0.000000,0.0,0.0,0.0
13671,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0


In [148]:
%%time
datos_estructurado_clas.to_sql(name="desc_1word_clas",con=prod,schema="public",if_exists = 'replace', method="multi",chunksize = 1000,index=True)
#datos_estructurado_clas.to_sql(name="lugar_1word_clas",con=prod,schema="public",if_exists = 'replace', method="multi",chunksize = 1000,index=True)

Wall time: 5.42 s
